In [ ]:
# ────────────────────────────────────────────────────────
# 라이브러리 / 상수 정의
# ────────────────────────────────────────────────────────
from functools import partial          # 콜백에 인자 “선바인딩” 하는 유틸
import queue                           # 스레드 간 안전한 FIFO 큐

from loguru import logger              # 구조화 로깅 라이브러리
import numpy as np                     # 수치 연산 / 버퍼 생성

# Hailo SDK: HEF 모델 파일, 가상 장치(VDevice) 및 스트림 포맷 정의
from hailo_platform import (
    HEF, VDevice,
    FormatType, HailoSchedulingAlgorithm,
)

# ────────────────────────────────────────────────────────
# HailoAsyncInference 클래스
# ────────────────────────────────────────────────────────
class HailoAsyncInference:
    """
    Hailo HEF 모델을 **비동기** 방식으로 실행해 스루풋을 최대화하는 헬퍼.

    ◼ 입력:
        이미 Log-Mel 스펙트로그램 형태로 전처리된 NumPy 텐서
        (예: (1, 64, 101, 1) UINT8 또는 (1, 64, 101, 1) FLOAT32)

    ◼ 출력:
        • 단일 출력 → NumPy 배열  
        • 다중 출력 → {출력_이름: NumPy 배열} 딕셔너리

    ◼ 특징:
        1) **Round-Robin 스케줄러**로 여러 작업을 HW 파이프라인에 고르게 배치
        2) **큐 기반 Producer/Consumer** 구조 — I/O·전처리를 별도 스레드에서 실행 가능
        3) SDK 가 제공하는 **run_async + callback** 패턴 사용
    """

    # 1) 초기화 --------------------------------------------------------------
    def __init__(
        self,
        hef_path,              # HEF 파일 경로 (컴파일된 Hailo 모델)
        input_queue,           # Producer 가 채우는 입력 텐서 큐
        output_queue,          # Consumer 가 읽을 결과 큐
        batch_size=1,          # 모델 추론 배치 크기(K=1 권장: 스트리밍 지연 최소화)
        input_type=None,       # 입력 스트림 타입(예: 'UINT8') — None 이면 HEF 메타 그대로
        output_type=None,      # {"out0": "UINT8", "out1": "FLOAT32"} 식 지정
        send_original_tensor=False  # 원본 텐서를 결과와 함께 저장(디버깅용)
    ):
        # 큐 핸들 저장
        self.input_queue = input_queue
        self.output_queue = output_queue

        # VDevice 생성 파라미터 — 스케줄러를 Round-Robin 으로 명시
        params = VDevice.create_params()
        params.scheduling_algorithm = HailoSchedulingAlgorithm.ROUND_ROBIN

        # HEF 로딩 후 가상 디바이스(VDevice) 인스턴스화
        self.hef = HEF(hef_path)
        self.target = VDevice(params)      # 실제 Hailo-8 장치를 추상화한 컨텍스트

        # HEF 에서 추론 모델(인터프리터) 생성
        self.infer_model = self.target.create_infer_model(hef_path)
        self.infer_model.set_batch_size(batch_size)  # 배치 사이즈 반영

        # 사용자가 타입을 오버라이드 했다면 VStream 포맷 변경
        if input_type:
            self._set_input_type(input_type)
        if output_type:
            self._set_output_type(output_type)

        # 기타 옵션 저장
        self.output_type = output_type
        self.send_original_tensor = send_original_tensor

    # 2) VStream 형식 설정 ---------------------------------------------------
    def _set_input_type(self, input_type):
        """
        모델에 입력 스트림이 1개뿐이라는 전제를 두고,  
        모든 입력의 FormatType 을 동일 값으로 덮어쓴다.
        """
        self.infer_model.input().set_format_type(getattr(FormatType, input_type))

    def _set_output_type(self, output_type_dict):
        """
        output_type_dict 예시:
            {"softmax": "UINT8"}  또는 {"cls": "FLOAT32", "score": "UINT8"}
        각 출력의 FormatType 을 개별적으로 설정한다.
        """
        for name, fmt in output_type_dict.items():
            self.infer_model.output(name).set_format_type(getattr(FormatType, fmt))

    # 3) 콜백 ---------------------------------------------------------------
    def callback(self, completion_info, bindings_list, batch_tensors):
        """
        run_async 작업이 완료될 때 SDK 가 호출하는 함수.

        • completion_info: 성공 여부 및 예외 정보
        • bindings_list  : run_async 에 전달했던 바인딩 객체 리스트
        • batch_tensors  : 해당 배치의 입력(디버깅용으로 큐에 다시 넘김)
        """
        if completion_info.exception:
            logger.error(f"Inference error: {completion_info.exception}")
            return  # 예외 발생 시 결과 무시

        # 각 바인딩마다 출력 버퍼 추출
        for i, bindings in enumerate(bindings_list):
            if len(bindings._output_names) == 1:    # 단일 출력
                result = bindings.output().get_buffer()
            else:                                   # 다중 출력
                result = {
                    n: np.expand_dims(bindings.output(n).get_buffer(), 0)
                    for n in bindings._output_names
                }
            # (입력, 출력) 튜플을 출력 큐에 push — Consumer 측 후처리용
            self.output_queue.put((batch_tensors[i], result))

    # 4) 정보 조회 -----------------------------------------------------------
    def get_vstream_info(self):
        """(입력 VStream 리스트, 출력 VStream 리스트) 반환."""
        return self.hef.get_input_vstream_infos(), self.hef.get_output_vstream_infos()

    def get_input_shape(self):
        """모델 첫 입력 VStream 의 텐서 shape 반환."""
        return self.hef.get_input_vstream_infos()[0].shape

    # 5) 메인 루프 -----------------------------------------------------------
    def run(self):
        """
        • `configure()` 컨텍스트 진입 → VStreams 실체화  
        • 입력 큐에서 배치를 꺼내 run_async 호출  
        • sentinel(None) 을 만나면 루프 종료 후 마지막 job.wait()
        """
        with self.infer_model.configure() as cfg:
            while True:
                batch_data = self.input_queue.get()         # 큐에서 다음 배치 pop
                if batch_data is None:                      # None = 종료 시그널
                    break

                # 원본 텐서 보존 여부에 따라 구조 결정
                tensors = batch_data if not self.send_original_tensor else batch_data[1]

                # 각 텐서마다 바인딩 객체 생성
                bindings_list = []
                for t in tensors:
                    b = self._create_bindings(cfg)          # 출력 버퍼 구성
                    b.input().set_buffer(np.asarray(t))     # 입력 버퍼 지정
                    bindings_list.append(b)

                # 디바이스가 Async 실행 가능할 때까지 대기
                cfg.wait_for_async_ready(timeout_ms=10_000)

                # 비동기 실행 → 완료 시 self.callback 호출
                job = cfg.run_async(
                    bindings_list,
                    partial(
                        self.callback,
                        batch_tensors=(
                            batch_data[0] if self.send_original_tensor else tensors
                        ),
                        bindings_list=bindings_list,
                    ),
                )
            # 마지막 비동기 잡이 완전히 끝날 때까지 블록
            job.wait(10_000)

    # 6) 내부 유틸 -----------------------------------------------------------
    def _dtype_str(self, info):
        """
        VStream 의 FormatType → NumPy dtype 문자열(e.g. 'uint8') 로 변환.
        output_type 사전이 없으면 HEF 메타데이터를 그대로 사용.
        """
        if self.output_type is None:
            return str(info.format.type).split(".")[1].lower()
        return self.output_type[info.name].lower()

    def _create_bindings(self, cfg):
        """
        • 출력 버퍼(NumPy 배열) 사전 준비
        • cfg.create_bindings() 로 바인딩 객체 생성
        """
        if self.output_type is None:
            # HEF 에 정의된 출력 스트림 형식(먼저 추론) 사용
            out_bufs = {
                info.name: np.empty(
                    self.infer_model.output(info.name).shape,     # 출력 shape
                    dtype=getattr(np, self._dtype_str(info)),      # np.uint8 등
                )
                for info in self.hef.get_output_vstream_infos()
            }
        else:
            # 사용자가 dtype 을 명시했을 때
            out_bufs = {
                n: np.empty(
                    self.infer_model.output(n).shape,
                    dtype=getattr(np, dt.lower()),
                )
                for n, dt in self.output_type.items()
            }
        return cfg.create_bindings(output_buffers=out_bufs)

# ────────────────────────────────────────────────────────
# Log-Mel 전용 유틸리티
# ────────────────────────────────────────────────────────
def validate_logmels(tensors, batch_size):
    """
    입력 텐서 리스트와 배치 크기 유효성 검증.

    • 텐서가 0개면 ValueError  
    • 텐서 수 % batch_size != 0 이면 ValueError
    """
    if not tensors:
        raise ValueError("Log-Mel 텐서 리스트가 비어 있습니다.")
    if len(tensors) % batch_size != 0:
        raise ValueError("배치 크기로 나누어떨어지지 않습니다.")

def divide_to_batches(tensors, batch_size):
    """
    제너레이터 패턴으로 배치 분할.
    메모리 복사 없이 리스트 슬라이스 view 를 반환하므로 가벼움.
    """
    for i in range(0, len(tensors), batch_size):
        yield tensors[i : i + batch_size]
